# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as mse

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


In [5]:
data = data.drop(['calls'], axis=1) # минуты мне кажутся ценнее если все-таки удалять

## Разбейте данные на выборки

In [6]:
features = data.drop(['is_ultra'], axis = 1)
target = data['is_ultra']

features_train, features_x, target_train, target_x = train_test_split(
    features, target, test_size=0.4, random_state=1)

features_test, features_valid, target_test, target_valid = train_test_split(
    features_x, target_x, test_size=0.5, random_state=1)

## Исследуйте модели

Начнем с дерева решений.

In [7]:
tree = DecisionTreeClassifier(random_state=1)
tree.fit(features_train, target_train)
tree_pred = tree.predict(features_valid)
ac_tree = accuracy_score(target_valid, tree_pred)

print("Качество модели:", ac_tree)

Качество модели: 0.7091757387247278


Попробуем больше деревьев.

In [8]:
best_model_forest = None
best_result_forest = 0
best_est = 0
for est in range(1, 30):
    forest = RandomForestClassifier(random_state=1, n_estimators=est)
    forest.fit(features_train, target_train)
    forest_pred = forest.predict(features_valid)
    ac_forest = accuracy_score(target_valid, forest_pred)
    if ac_forest > best_result_forest:
        best_model_forest = forest
        best_result_forest = ac_forest
        best_est = est
    
print("Качество модели:", best_result_forest, ", количество деревьев:", best_est)

Качество модели: 0.8009331259720062 , количество деревьев: 28


Пойдем еще к соседям.

In [9]:
best_model_neib = None
best_result_neib = 0
best_neib = 0
for neib in range(1, 30):
    kn = KNeighborsClassifier(n_neighbors=neib)
    kn.fit(features_train, target_train)
    kn_pred = kn.predict(features_valid)
    ac_kn = accuracy_score(target_valid, kn_pred)
    if ac_kn > best_result_neib:
        best_model_neib = kn
        best_result_neib = ac_kn
        best_neib = neib

print("Качество модели:", best_result_neib, ", количество соседей:", best_neib)

Качество модели: 0.7713841368584758 , количество соседей: 11


Вывод - пока в лесу живется лучше всего.

## Проверьте модель на тестовой выборке

In [11]:
forest_test = RandomForestClassifier(random_state=1, n_estimators=28)
forest_test.fit(features_train, target_train)
forest_test_pred = forest_test.predict(features_test)
ac_forest_test = accuracy_score(target_test, forest_test_pred)
print("Качество модели:", ac_forest_test)

Качество модели: 0.7729393468118196


## (бонус) Проверьте модели на адекватность

In [15]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(random_state=1, strategy="most_frequent")
dummy.fit(features_train, target_train)
dummy_pred = dummy.predict(features_test)
ac_dummy = accuracy_score(target_test, dummy_pred)
print("Качество модели:", ac_dummy)

Качество модели: 0.6640746500777605


## Вывод

Мы проверили качество работы нескольких моделей и пришли к выводу что для достижения лучшего результата лесные барышни самое то! На них и будет подниматся наша новая 